In [26]:
import os

import numpy as np
import imageio.v3 as iio
import pandas as pd
import json
import cv2
import sklearn
from skimage import io
from sklearn.model_selection import train_test_split
import cv2

# tf imports
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.layers import Dense
from keras import backend as K


In [27]:
def depth_image_to_grayscale(img_path):
    depth_image = iio.imread(img_path)
    depth_instensity = np.array(256 * depth_image / 0x0fff,dtype=np.uint8)
    iio.imwrite('../data/GrayScaleImages/grayscale_1.png', depth_instensity)


In [28]:
depth_image_to_grayscale("..\data\RGBImages\RGB_268.png")




In [29]:
img = cv2.imread("..\data\RGBImages\RGB_268.png")
print(img)

[[[223 221 209]
  [222 220 208]
  [222 220 208]
  ...
  [138 129 111]
  [135 128 110]
  [135 128 110]]

 [[222 220 208]
  [222 220 208]
  [222 220 208]
  ...
  [138 130 109]
  [135 129 108]
  [135 129 108]]

 [[222 219 209]
  [221 218 208]
  [220 217 207]
  ...
  [136 130 109]
  [135 129 108]
  [135 129 108]]

 ...

 [[151 170 164]
  [151 170 164]
  [147 166 160]
  ...
  [ 64  72  60]
  [ 65  73  61]
  [ 66  74  62]]

 [[151 170 164]
  [150 169 163]
  [147 166 160]
  ...
  [ 64  72  60]
  [ 65  73  61]
  [ 66  74  62]]

 [[151 170 164]
  [150 169 163]
  [147 166 160]
  ...
  [ 65  73  61]
  [ 66  74  62]
  [ 68  76  64]]]


In [32]:

# RGB images are to dirty to use threshold and grayscale

img = cv2.imread('..\data\RGBImages\RGB_268.png')

# image resize and down-size to focus on object
w, h, c = img.shape
img = img[ (w // 2 - 500 ) : (w // 2 + 500) , (h // 2 - 500 + 100)  : (h // 2 + 500 + 100) ]
rsz_img = cv2.resize(img, None, fx=0.50, fy=0.50)
# concert image from RGB to Grayscale
gray = cv2.cvtColor(rsz_img, cv2.COLOR_BGR2GRAY)

# Threshold fine-tuning
retval, thresh_gray = cv2.threshold(gray, thresh=100, maxval=255, type=cv2.THRESH_BINARY)


cv2.imshow("resized gray", thresh_gray)
cv2.waitKey(0)

# # find where the signature is and make a cropped region
points = np.argwhere(thresh_gray==0) # find where the black pixels are
points = np.fliplr(points) # store them in x,y coordinates instead of row,col indices
# points
# x, y, w, h = cv2.boundingRect(points) # create a rectangle around those points
# x, y, w, h = x-10, y-10, w+20, h+20 # make the box a little bigger
# crop = gray[y:y+h, x:x+w] # create a cropped region of the gray image
#
# # get the thresholded crop
# retval, thresh_crop = cv2.threshold(crop, thresh=200, maxval=255, type=cv2.THRESH_BINARY)
#
# # display
# cv2.imshow("Cropped and thresholded image", thresh_crop)
# cv2.waitKey(0)
#
# cv2.imwrite('Prop_268.png', thresh_crop)

In [33]:
import cv2
import numpy as np
# get object region by using threshold and depth images


# load images
img_depth = cv2.imread("..\data\DepthImages\Depth_150.png", -1)
# also providing the RGB image because depth image can't really be seen
img_rgb = cv2.imread("..\data\RGBImages\RGB_150.png")

# check if both images have the same shape and size, channels don't matter
print(f"Image shape depth: {img_depth.shape}")
print(f"Image shape RGB: {img_rgb.shape}")

w, h, c = img_rgb.shape
# cropping
img_rgb = img_rgb[ (w // 2 - 500 ) : (w // 2 + 500) , (h // 2 - 500 + 100)  : (h // 2 + 500 + 100) ]
img_depth = img_depth[ (w // 2 - 500 ) : (w // 2 + 500) , (h // 2 - 500 + 100)  : (h // 2 + 500 + 100) ]
# zoom
img_rgb = cv2.resize(img_rgb, None, fx=0.50, fy=0.50)
img_depth = cv2.resize(img_depth, None, fx=0.50, fy=0.50)

print(np.amax(img_depth))

# finding signature of the depth image and propose a region
## filling o with 1 because of some argwhere errors
img_depth[img_depth == 0 ] += 1390
points = np.argwhere(img_depth < 830) # find where the black pixels are
points = np.fliplr(points) # store them in x,y coordinates instead of row,col indices

points

#
# rsz_img = cv2.resize(img, None, fx=0.50, fy=0.50)
# # concert image from RGB to Grayscale
# gray = cv2.cvtColor(rsz_img, cv2.COLOR_BGR2GRAY)


Image shape depth: (1080, 1920)
Image shape RGB: (1080, 1920, 3)
1390


array([[  8,   0],
       [  9,   0],
       [  8,   3],
       ...,
       [266, 482],
       [267, 482],
       [281, 482]], dtype=int64)